In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import math
import cv2
import numpy as np
import csv
from pylab import *
from matplotlib import gridspec
import datetime
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
cdict = {'red': ((0.0, 1.0, 1.0),
                 (0.125, 1.0, 1.0),
                 (0.25, 1.0, 1.0),
                 (0.5625, 1.0, 1.0),
                 (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 0.0),
                   (0.25, 0.0, 0.0),
                   (0.5625, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),
         'blue': ((0.0, 0.0, 0.0),
                  (0.5, 0.0, 0.0),
                  (1.0, 0.0, 0.0))}
my_cmap = matplotlib.colors.LinearSegmentedColormap('my_colormap',cdict,256)
%matplotlib inline

In [129]:
import pickle
(samples,labels) = pickle.load( open( "data_new.p", "rb" ) )
print(samples.shape)
print(labels.shape)

(73014, 10, 5, 9)
(73014,)


In [130]:
from sklearn.utils import shuffle
def class_balance(samples,labels):
    samples0 = samples[labels==0,:,:,:]
    samples1 = samples[labels==1,:,:,:]
    selected = np.random.choice(len(samples0), len(samples1), replace=False)
    samples0 = samples0[selected,:,:,:]
    labels0 = np.zeros((len(samples0)))
    labels1 = np.ones((len(samples1)))
    samples = np.concatenate((samples0,samples1),axis=0)
    labels = np.concatenate((labels0,labels1),axis=0)
    samples, labels = shuffle(samples, labels)
    return samples, labels
samples, labels = class_balance(samples,labels)
one_hot_labels = np.zeros((labels.shape[0],2))
one_hot_labels[labels==0,0]=1
one_hot_labels[labels==1,1]=1
print(samples.shape)
print(one_hot_labels.shape)

(15840, 10, 5, 9)
(15840, 2)


In [138]:
def MinMax_Normalization(samples):
    samples_shape = samples.shape
    samples = np.reshape(samples,(samples_shape[0],samples_shape[1]*samples_shape[2]*samples_shape[3]))
    scaler = MinMaxScaler().fit(samples)
    samples_normalized = scaler.transform(samples)
    samples_normalized = np.reshape(samples_normalized,(samples_shape[0],samples_shape[1],samples_shape[2],samples_shape[3]))
    return samples_normalized, scaler
    
samples_scaled, scaler = MinMax_Normalization(samples)

In [139]:
def feature_select(samples_scaled,features):
    return samples_scaled[:,:,:,features]

features = [0,1,2,7,8]
samples_scaled = feature_select(samples_scaled,features)
print(samples_scaled.shape)

(15840, 10, 5, 5)


In [ ]:
from keras.models import Sequential
from keras.layers import Reshape, Dense, Convolution2D, Deconvolution2D, Flatten, Input, Dropout, MaxPooling2D, Activation
from keras.models import model_from_json
from keras.activations import relu, softmax, linear
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU

In [142]:
# deep CNN model

model = Sequential()
# 10*5*5
model.add(Convolution2D(16, 3, 3,input_shape=(10, 5, 5),border_mode='valid'))
model.add(ELU())
# 8*3*16
model.add(Convolution2D(32, 3, 3,border_mode='valid'))
model.add(ELU())
# 6*1*32
model.add(Flatten())
# 192
model.add(Dense(64))
model.add(ELU())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history_object = model.fit(samples_scaled, one_hot_labels, nb_epoch=100, batch_size=64, verbose=1)

Epoch 1/100
15840/15840 [==============================] - 1s - loss: 0.6139 - acc: 0.6599     
Epoch 2/100
15840/15840 [==============================] - 1s - loss: 0.5806 - acc: 0.6921     
Epoch 3/100
15840/15840 [==============================] - 1s - loss: 0.5746 - acc: 0.6955     
Epoch 4/100
15840/15840 [==============================] - 1s - loss: 0.5720 - acc: 0.7011     
Epoch 5/100
15840/15840 [==============================] - 1s - loss: 0.5707 - acc: 0.7008     
Epoch 6/100
15840/15840 [==============================] - 1s - loss: 0.5664 - acc: 0.7039     
Epoch 7/100
15840/15840 [==============================] - 1s - loss: 0.5637 - acc: 0.7032     
Epoch 8/100
15840/15840 [==============================] - 1s - loss: 0.5588 - acc: 0.7080     
Epoch 9/100
15840/15840 [==============================] - 1s - loss: 0.5583 - acc: 0.7085     
Epoch 10/100
15840/15840 [==============================] - 1s - loss: 0.5546 - acc: 0.7157     
Epoch 11/100
15840/15840 [=============

In [141]:
# deep ANN model

model = Sequential()
model.add(Flatten(input_shape=(10, 5, 5)))
# 192
model.add(Dense(64))
model.add(ELU())
model.add(Dense(32))
model.add(ELU())
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

history_object = model.fit(samples_scaled, one_hot_labels, nb_epoch=10, batch_size=64, verbose=1)

Epoch 1/10
15840/15840 [==============================] - 1s - loss: 0.6122 - acc: 0.6653     
Epoch 2/10
15840/15840 [==============================] - 0s - loss: 0.5772 - acc: 0.6956     
Epoch 3/10
15840/15840 [==============================] - 0s - loss: 0.5679 - acc: 0.7033     
Epoch 4/10
15840/15840 [==============================] - 0s - loss: 0.5653 - acc: 0.7029     
Epoch 5/10
15840/15840 [==============================] - 0s - loss: 0.5603 - acc: 0.7091     
Epoch 6/10
15840/15840 [==============================] - 0s - loss: 0.5590 - acc: 0.7107     
Epoch 7/10
15840/15840 [==============================] - 0s - loss: 0.5551 - acc: 0.7119     
Epoch 8/10
15840/15840 [==============================] - 0s - loss: 0.5514 - acc: 0.7159     
Epoch 9/10
15840/15840 [==============================] - 0s - loss: 0.5489 - acc: 0.7145     
Epoch 10/10
15840/15840 [==============================] - 0s - loss: 0.5492 - acc: 0.7128     


In [55]:
model.predict(samples_scaled[0:10,:,:,:])

array([[ 0.89720249,  0.10279749],
       [ 0.9130066 ,  0.08699342],
       [ 0.90403706,  0.09596292],
       [ 0.93120247,  0.06879755],
       [ 0.85641062,  0.14358936],
       [ 0.88387764,  0.1161223 ],
       [ 0.91124195,  0.08875811],
       [ 0.95300812,  0.04699193],
       [ 0.86156821,  0.13843176],
       [ 0.89209193,  0.10790803]], dtype=float32)

In [ ]:
def data_for_shallow
